In [13]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumBarunGothic'

In [14]:
# 전처리된 카카오맵 크롤링 데이터 불러오기
kakao_food = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_pre.csv", encoding='utf-8')
kakao_food

,Name,Category,Score,Review_Num,Link,Addr,ADDR_area
0,덕수식당,restaurant,4.7,670,https://place.map.kakao.com/8891473,충남 태안군 태안읍 중앙로 133-8 1층(지번) 태안읍 동문리 339-15,충청
1,한일식당,restaurant,3.6,347,https://place.map.kakao.com/9573537,충남 예산군 삽교읍 삽교역로 58(지번) 삽교읍 신가리 20-14,충청
2,장춘닭개장,restaurant,4.2,187,https://place.map.kakao.com/11169052,충남 당진시 정안로 50 1층(지번) 원당동 381,충청
3,엄가네본가시골집,restaurant,3.7,284,https://place.map.kakao.com/13311506,충남 천안시 서북구 봉정로 168-2(지번) 성정동 142-6,충청
4,에이스식당,restaurant,4.8,109,https://place.map.kakao.com/9250070,충남 당진시 면천면 면천서문1길 70(지번) 면천면 성상리 768-1,충청
...,...,...,...,...,...,...,...
5457,모드니,cafe,4.6,54,https://place.map.kakao.com/6376422,광주 동구 동명로14번길 19 1층(지번) 동명동 73-19,광주
5458,엄니도시락한식뷔페 평동점,restaurant,4.0,15,https://place.map.kakao.com/206831746,광주 광산구 평동산단로 192 1층(지번) 월전동 1208,광주
5459,한옥식당,restaurant,3.9,50,https://place.map.kakao.com/21301951,광주 남구 백서로 87-2(지번) 양림동 137,광주
5460,프랭크버거 광주상무점,restaurant,3.9,29,https://place.map.kakao.com/1113376476,광주 서구 상무중앙로 29 1층(지번) 치평동 1178-6,광주


In [15]:
kakao_food["Category"].unique()

array(['restaurant', 'cafe'], dtype=object)

In [16]:
kakao_food = kakao_food.rename(columns={
    'Name' : "식당이름",
    "Category" : "카테고리",
    "Score" : "별점",
    "Review_Num" : "리뷰 수",
    "Link" : "웹사이트",
    "Addr" : "주소",
    "ADDR_area" : "지역명"
})
kakao_food.head()

,식당이름,카테고리,별점,리뷰 수,웹사이트,주소,지역명
0,덕수식당,restaurant,4.7,670,https://place.map.kakao.com/8891473,충남 태안군 태안읍 중앙로 133-8 1층(지번) 태안읍 동문리 339-15,충청
1,한일식당,restaurant,3.6,347,https://place.map.kakao.com/9573537,충남 예산군 삽교읍 삽교역로 58(지번) 삽교읍 신가리 20-14,충청
2,장춘닭개장,restaurant,4.2,187,https://place.map.kakao.com/11169052,충남 당진시 정안로 50 1층(지번) 원당동 381,충청
3,엄가네본가시골집,restaurant,3.7,284,https://place.map.kakao.com/13311506,충남 천안시 서북구 봉정로 168-2(지번) 성정동 142-6,충청
4,에이스식당,restaurant,4.8,109,https://place.map.kakao.com/9250070,충남 당진시 면천면 면천서문1길 70(지번) 면천면 성상리 768-1,충청


#### 카카오맵 지오코딩 -> "주소" 열을 통해 위도, 경도 알아내기 

In [17]:
import requests
import numpy as np

def get_coordinate(address):  # 주소를 입력받음
    rest_api_key = '19686e2baf6e34febce26badd260a1a7'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    headers = {'Authorization': f'KakaoAK {rest_api_key}'}
    params = {'query': address}

    try:
        r = requests.get(url, headers=headers, params=params, timeout=3)
        if r.status_code == 200:
            documents = r.json().get('documents', [])
            if documents:
                addr_info = documents[0].get('address')
                if addr_info:
                    return float(addr_info['y']), float(addr_info['x'])  # (위도, 경도)
        return (None, None)
    except Exception as e:
        print(f"오류 발생: {e} / 주소: {address}")
        return (None, None)



In [18]:
from tqdm import tqdm
import time

# 진행
coordinates = []

for addr in tqdm(kakao_food['주소']):
    coord = get_coordinate(addr)
    coordinates.append(coord)
    time.sleep(0.5)

# 결과 저장
kakao_food['위도'] = [lat for lat, _ in coordinates]
kakao_food['경도'] = [lon for _, lon in coordinates]


 96%|█████████▋| 5260/5462 [52:26<02:01,  1.67it/s] 

오류 발생: HTTPSConnectionPool(host='dapi.kakao.com', port=443): Max retries exceeded with url: /v2/local/search/address.json?query=%EA%B4%91%EC%A3%BC+%EC%84%9C%EA%B5%AC+%EB%82%B4%EB%B0%A9%EB%A1%9C+356+1%EC%B8%B5%28%EC%A7%80%EB%B2%88%29+%ED%99%94%EC%A0%95%EB%8F%99+23-51 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024E1C43D7F0>, 'Connection to dapi.kakao.com timed out. (connect timeout=3)')) / 주소: 광주 서구 내방로 356 1층(지번) 화정동 23-51


100%|██████████| 5462/5462 [54:30<00:00,  1.67it/s]


In [19]:
kakao_food

,식당이름,카테고리,별점,리뷰 수,웹사이트,주소,지역명,위도,경도
0,덕수식당,restaurant,4.7,670,https://place.map.kakao.com/8891473,충남 태안군 태안읍 중앙로 133-8 1층(지번) 태안읍 동문리 339-15,충청,36.751762,126.303150
1,한일식당,restaurant,3.6,347,https://place.map.kakao.com/9573537,충남 예산군 삽교읍 삽교역로 58(지번) 삽교읍 신가리 20-14,충청,36.684627,126.749091
2,장춘닭개장,restaurant,4.2,187,https://place.map.kakao.com/11169052,충남 당진시 정안로 50 1층(지번) 원당동 381,충청,36.906543,126.649845
3,엄가네본가시골집,restaurant,3.7,284,https://place.map.kakao.com/13311506,충남 천안시 서북구 봉정로 168-2(지번) 성정동 142-6,충청,36.816957,127.142706
4,에이스식당,restaurant,4.8,109,https://place.map.kakao.com/9250070,충남 당진시 면천면 면천서문1길 70(지번) 면천면 성상리 768-1,충청,36.816980,126.666814
...,...,...,...,...,...,...,...,...,...
5457,모드니,cafe,4.6,54,https://place.map.kakao.com/6376422,광주 동구 동명로14번길 19 1층(지번) 동명동 73-19,광주,35.150059,126.925408
5458,엄니도시락한식뷔페 평동점,restaurant,4.0,15,https://place.map.kakao.com/206831746,광주 광산구 평동산단로 192 1층(지번) 월전동 1208,광주,35.123978,126.768528
5459,한옥식당,restaurant,3.9,50,https://place.map.kakao.com/21301951,광주 남구 백서로 87-2(지번) 양림동 137,광주,35.140323,126.915118
5460,프랭크버거 광주상무점,restaurant,3.9,29,https://place.map.kakao.com/1113376476,광주 서구 상무중앙로 29 1층(지번) 치평동 1178-6,광주,35.149591,126.847908


In [ ]:
#kakao_food.to_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_geocode.csv", index=False, encoding="utf-8-sig")

* 지오코딩에 실패한 데이터 건수 : 229건 -> 재진행


In [ ]:
kakao_food["위도"].isnull().sum()

229

In [ ]:
failed_geocoding = kakao_food[kakao_food["위도"].isnull()]
failed_geocoding

,식당이름,카테고리,별점,리뷰 수,웹사이트,주소,지역명,위도,경도
30,블렌데렌,cafe,4.6,154,https://place.map.kakao.com/853217400,"충남 천안시 동남구 먹거리8길 5 1, 2층(지번) 신부동 452-10",충청,NaN,NaN
55,조리고 천안점,restaurant,4.3,192,https://place.map.kakao.com/549841262,"충남 천안시 동남구 차돌고개7길 1 2, 3층(지번) 다가동 376-4",충청,NaN,NaN
65,삿뽀로 천안점,restaurant,3.8,48,https://place.map.kakao.com/12515606,충남 천안시 서북구 원두정1길 7 1-2층(지번) 두정동 807,충청,NaN,NaN
76,삿뽀로 천안점,restaurant,3.8,48,https://place.map.kakao.com/12515606,충남 천안시 서북구 원두정1길 7 1-2층(지번) 두정동 807,충청,NaN,NaN
152,숲속의문,cafe,5.0,91,https://place.map.kakao.com/432996016,충남 천안시 동남구 양지말2길 61 1-3층(지번) 유량동 82-8,충청,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5352,투썸플레이스 광주풍암점,cafe,3.6,4,https://place.map.kakao.com/26774977,광주 서구 풍암순환로 113 1-2층(지번) 풍암동 1091-1,광주,NaN,NaN
5360,보성녹돈,restaurant,4.6,21,https://place.map.kakao.com/8395711,"광주 서구 내방로 242 1,2층(지번) 쌍촌동 266-1",광주,NaN,NaN
5370,필링돈까스 본점,restaurant,3.7,247,https://place.map.kakao.com/14507290,광주 북구 설죽로 297 2-3층(지번) 용봉동 1129-1,광주,NaN,NaN
5385,크라비앙코,cafe,4.4,30,https://place.map.kakao.com/26640525,광주 동구 지호로 132 1-2층(지번) 지산동 111-8,광주,NaN,NaN


In [35]:
from tqdm import tqdm
import time
import re

# 주소 전처리 함수: 괄호 제거 + 양쪽 공백 제거
def clean_address(addr):
    return re.sub(r"\(.*?\)", "", addr).strip()

# 실패한 인덱스만 추출
failed_idx = kakao_food[kakao_food["위도"].isnull()].index

# 다시 시도할 좌표 저장
retry_coordinates = {}
failed_again = []

# 재시도 루프
for idx in tqdm(failed_idx):
    raw_addr = kakao_food.loc[idx, "주소"]
    clean_addr = clean_address(raw_addr)
    coord = get_coordinate(clean_addr)
    if isinstance(coord, tuple) and coord is not np.nan:
        retry_coordinates[idx] = coord
    else:
        failed_again.append((idx, clean_addr))  # 실패한 주소 저장
    time.sleep(0.5)

100%|██████████| 228/228 [02:15<00:00,  1.68it/s]


In [36]:
# 값 대입
for idx, (lat, lon) in retry_coordinates.items():
    kakao_food.at[idx, "위도"] = lat
    kakao_food.at[idx, "경도"] = lon

# 결과 출력
print("✅ 재시도 후 위도 누락 건수:", kakao_food["위도"].isnull().sum())
print("✅ 재시도 후 경도 누락 건수:", kakao_food["경도"].isnull().sum())
print(f"❌ 여전히 실패한 주소 개수: {len(failed_again)}")

✅ 재시도 후 위도 누락 건수: 228
✅ 재시도 후 경도 누락 건수: 228
❌ 여전히 실패한 주소 개수: 0


* 우회 지오코딩 코드 : 키워드 검색 방식(식당이름 + 지역명)

In [42]:
from tqdm import tqdm
import time

# 실패한 인덱스 추출
failed_final = kakao_food[kakao_food["위도"].isnull()].index

# 우회 지오코딩 함수: 식당이름 + 지역명
def get_coordinate_by_keyword(place_name, region):
    rest_api_key = '19686e2baf6e34febce26badd260a1a7'
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    headers = {'Authorization': f'KakaoAK {rest_api_key}'}
    params = {'query': f"{region} {place_name}"}

    try:
        r = requests.get(url, headers=headers, params=params, timeout=3)
        if r.status_code == 200:
            documents = r.json().get('documents', [])
            if documents:
                top = documents[0]
                return float(top['y']), float(top['x'])
        return np.nan
    except Exception as e:
        print(f"❌ 키워드 우회 실패: {e} / 검색어: {region} {place_name}")
        return np.nan

# 우회 시도
for idx in tqdm(failed_final):
    name = kakao_food.loc[idx, "식당이름"]
    region = kakao_food.loc[idx, "지역명"]
    coord = get_coordinate_by_keyword(name, region)

    if isinstance(coord, tuple) and coord is not np.nan:
        kakao_food.at[idx, "위도"] = coord[0]
        kakao_food.at[idx, "경도"] = coord[1]
    else:
        print(f"❌ 최종 실패: [{idx}] {region} {name}")
    time.sleep(0.5)

# 결과 확인
print("✅ 최종 위도 결측 수:", kakao_food["위도"].isnull().sum())
print("✅ 최종 경도 결측 수:", kakao_food["경도"].isnull().sum())


 54%|█████▎    | 122/228 [01:12<01:02,  1.69it/s]

❌ 최종 실패: [2920] 전라 봉가면옥


 54%|█████▍    | 123/228 [01:12<01:02,  1.69it/s]

❌ 최종 실패: [2925] 전라 원조연기식당


 54%|█████▍    | 124/228 [01:13<01:01,  1.69it/s]

...

❌ 최종 실패: [5024] 경상 보움


 93%|█████████▎| 211/228 [02:05<00:10,  1.70it/s]

❌ 최종 실패: [5025] 경상 버터앤브루


100%|██████████| 228/228 [02:15<00:00,  1.68it/s]

✅ 최종 위도 결측 수: 21
✅ 최종 경도 결측 수: 21


In [ ]:
from tqdm import tqdm
import time

# 진행
coordinates = []

for addr in tqdm(kakao_food['주소']):
    coord = get_coordinate(addr)
    coordinates.append(coord)
    time.sleep(0.5)

# 결과 저장
kakao_food['위도'] = [lat for lat, _ in coordinates]
kakao_food['경도'] = [lon for _, lon in coordinates]


 96%|█████████▋| 5260/5462 [52:26<02:01,  1.67it/s] 

오류 발생: HTTPSConnectionPool(host='dapi.kakao.com', port=443): Max retries exceeded with url: /v2/local/search/address.json?query=%EA%B4%91%EC%A3%BC+%EC%84%9C%EA%B5%AC+%EB%82%B4%EB%B0%A9%EB%A1%9C+356+1%EC%B8%B5%28%EC%A7%80%EB%B2%88%29+%ED%99%94%EC%A0%95%EB%8F%99+23-51 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024E1C43D7F0>, 'Connection to dapi.kakao.com timed out. (connect timeout=3)')) / 주소: 광주 서구 내방로 356 1층(지번) 화정동 23-51


100%|██████████| 5462/5462 [54:30<00:00,  1.67it/s]


In [43]:
kakao_food["위도"].isnull().sum()

21

In [2]:
import pandas as pd

kakao_food = pd.read_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_geocode.csv", encoding='utf-8')
kakao_food

,식당이름,카테고리,별점,리뷰 수,웹사이트,주소,지역명,위도,경도
0,덕수식당,restaurant,4.7,670,https://place.map.kakao.com/8891473,충남 태안군 태안읍 중앙로 133-8 1층(지번) 태안읍 동문리 339-15,충청,36.751762,126.303150
1,한일식당,restaurant,3.6,347,https://place.map.kakao.com/9573537,충남 예산군 삽교읍 삽교역로 58(지번) 삽교읍 신가리 20-14,충청,36.684627,126.749091
2,장춘닭개장,restaurant,4.2,187,https://place.map.kakao.com/11169052,충남 당진시 정안로 50 1층(지번) 원당동 381,충청,36.906543,126.649845
3,엄가네본가시골집,restaurant,3.7,284,https://place.map.kakao.com/13311506,충남 천안시 서북구 봉정로 168-2(지번) 성정동 142-6,충청,36.816957,127.142706
4,에이스식당,restaurant,4.8,109,https://place.map.kakao.com/9250070,충남 당진시 면천면 면천서문1길 70(지번) 면천면 성상리 768-1,충청,36.816980,126.666814
...,...,...,...,...,...,...,...,...,...
5457,모드니,cafe,4.6,54,https://place.map.kakao.com/6376422,광주 동구 동명로14번길 19 1층(지번) 동명동 73-19,광주,35.150059,126.925408
5458,엄니도시락한식뷔페 평동점,restaurant,4.0,15,https://place.map.kakao.com/206831746,광주 광산구 평동산단로 192 1층(지번) 월전동 1208,광주,35.123978,126.768528
5459,한옥식당,restaurant,3.9,50,https://place.map.kakao.com/21301951,광주 남구 백서로 87-2(지번) 양림동 137,광주,35.140323,126.915118
5460,프랭크버거 광주상무점,restaurant,3.9,29,https://place.map.kakao.com/1113376476,광주 서구 상무중앙로 29 1층(지번) 치평동 1178-6,광주,35.149591,126.847908


In [3]:
failed_geocoding = kakao_food[kakao_food["위도"].isnull()]
failed_geocoding

,식당이름,카테고리,별점,리뷰 수,웹사이트,주소,지역명,위도,경도
2920,봉가면옥,restaurant,4.2,218,https://place.map.kakao.com/20169194,전북특별자치도 남원시 소리길 177 1-2층(지번) 신촌동 410-3,전라,NaN,NaN
2925,원조연기식당,restaurant,3.6,38,https://place.map.kakao.com/8445922,"전북특별자치도 고창군 아산면 선운대로 2727 1,2층(지번) 아산면 삼인리 29-29",전라,NaN,NaN
2933,봉가면옥,restaurant,4.2,218,https://place.map.kakao.com/20169194,전북특별자치도 남원시 소리길 177 1-2층(지번) 신촌동 410-3,전라,NaN,NaN
2938,원조연기식당,restaurant,3.6,38,https://place.map.kakao.com/8445922,"전북특별자치도 고창군 아산면 선운대로 2727 1,2층(지번) 아산면 삼인리 29-29",전라,NaN,NaN
2948,미소식당,restaurant,4.7,216,https://place.map.kakao.com/95562205,전북특별자치도 순창군 순창읍 장류로 290 1-2층(지번) 순창읍 순화리 471-4,전라,NaN,NaN
3077,새만금횟집,restaurant,3.9,129,https://place.map.kakao.com/21483325,전북특별자치도 군산시 비응안7길 13 1-2층(지번) 비응도동 62-12,전라,NaN,NaN
4717,헤이안,cafe,3.8,681,https://place.map.kakao.com/464239111,"경북 포항시 북구 해안로 219 1,2층(지번) 두호동 156",경상,NaN,NaN
4750,카페 CW,restaurant,4.3,350,https://place.map.kakao.com/2058980864,경북 경산시 강변동로 290 1-3층(지번) 대평동 437-1,경상,NaN,NaN
4753,야드,cafe,4.4,347,https://place.map.kakao.com/940762021,경북 경주시 천군2길 2 1-2층(지번) 천군동 431-7,경상,NaN,NaN
4759,갈비만 안동점,restaurant,3.7,25,https://place.map.kakao.com/759688755,"경북 안동시 옥동길 56 1, 2층(지번) 옥동 944-1",경상,NaN,NaN


In [4]:
print("제거 전:", len(kakao_food))
kakao_food = kakao_food.dropna(subset=["위도", "경도"])
print("제거 후:", len(kakao_food))

제거 전: 5462
제거 후: 5441


In [5]:
kakao_food.to_csv(f"C:/Users/mcw08/OneDrive/바탕 화면/동덕여대/2025 문화 디지털혁신 및 데이터 활용 공모전/새로운 데이터/카카오맵 맛집 크롤링_전처리_데이터/kakao_food_geocode.csv", index=False, encoding="utf-8-sig")